In [1]:
from geopyspark.geopycontext import GeoPyContext
from geopyspark.geotrellis.constants import SPATIAL
from geopyspark.geotrellis.geotiff_rdd import geotiff_rdd
from geopyspark.geotrellis.tile_layer import collect_pyramid_metadata, tile_to_layout, pyramid, reproject
from geopyspark.geotrellis.catalog import write, read

In [2]:
from pyspark import SparkContext

In [3]:
sc = SparkContext(appName="Ingest")

In [4]:
geopysc = GeoPyContext(sc)

In [5]:
path = "hdfs:/rasters/Globe15_TIFF.tif"

In [6]:
rdd = geotiff_rdd(geopysc, SPATIAL, path, maxTileSize=256)

In [7]:
rdd.first()

({'epsg': 4326,
  'extent': {'xmax': -48.8286849246169,
   'xmin': -48.8642158112729,
   'ymax': -5.28478156613762,
   'ymin': -5.32031245279362},
  'proj4': None},
 {'data': array([[[ 34,  47,  89, ..., 151, 129, 123],
          [ 28,  41,  84, ..., 146, 118, 120],
          [ 57,  69, 104, ..., 139, 102, 112],
          ..., 
          [108, 112, 104, ...,  36,  28,  38],
          [125, 111, 104, ...,  23,  31,  48],
          [109,  81,  83, ...,  41,  67,  81]],
  
         [[104, 109, 138, ..., 152, 131, 125],
          [ 94, 101, 133, ..., 148, 120, 122],
          [116, 124, 151, ..., 142, 104, 112],
          ..., 
          [108, 115, 111, ...,  74,  61,  66],
          [125, 115, 112, ...,  63,  65,  76],
          [112,  87,  91, ...,  79, 101, 111]],
  
         [[ 62,  77, 119, ..., 136, 118, 115],
          [ 52,  69, 115, ..., 132, 107, 112],
          [ 76,  94, 134, ..., 126,  92, 102],
          ..., 
          [ 94, 105, 105, ...,  34,  24,  32],
          [111, 105

In [11]:
(_, metadata) = collect_pyramid_metadata(geopysc, SPATIAL, rdd, crs="EPSG:4326", tile_size=256)

In [12]:
laid_out = tile_to_layout(geopysc, SPATIAL, rdd, metadata)

In [13]:
pyramided = pyramid(geopysc, SPATIAL, laid_out, metadata, 256, 11, 0)

In [18]:
for zoom, layer_rdd, layer_metadata in pyramided:
    write(geopysc, SPATIAL, "hdfs:///catalog", "python-benchmark", zoom, layer_rdd, layer_metadata)

Py4JError: An error occurred while calling z:geopyspark.geotrellis.io.AttributeStoreFactory.buildHadoop. Trace:
py4j.Py4JException: Method buildHadoop([class java.lang.String]) does not exist
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:318)
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:339)
	at py4j.Gateway.invoke(Gateway.java:274)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:214)
	at java.lang.Thread.run(Thread.java:745)

